In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D, Concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import glob
import os
from PIL import Image
import pandas as pd
from keras_vggface.vggface import VGGFace
import tensorflow as tf
from keras import regularizers

In [2]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT)
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
X = []
Y = []
# 建立ID
ID_list = []
# root = "C:\workspace\server"
root = os.getcwd()
# 所有圖片的路徑 "C:\workspace\python_workspace\DeepLearning\detected_face\*"
users = glob.glob(os.path.join(root,"private","users_face","*"))
users
# for img_path in all_images_path:
#     # 取得學生id
#     id = img_path.split("\\")[-2]
#     img = np.array(Image.open(img_path).resize(IMG_SIZE).convert("RGB"))
#     X.append(img)
#     Y.append(name_map[id])
# X = np.array(X)
# Y = np.array(Y)
# Y = to_categorical(Y)
# # 輸出層的結果數
# out_dim = Y.shape[1]

[]

In [3]:
inputt = Input(shape=IMG_SHAPE)

base_model = VGGFace(model='resnet50', include_top=False)

for x in base_model.layers:
    x.trainable = False

# 將(224,224,3)的張量當作輸入數據傳給 base_model
# 得到 x 為 base_model 最後一層(AveragePooling2D)輸出的特徵
x = base_model(inputt)
x = Flatten()(x)
x = Dense(2048, activation="relu")(x)
x = Dense(1024, activation="relu")(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
x = Dense(128, activation="relu")(x)
model = Model(inputs=inputt, outputs=x)

output = Dense(out_dim, activation='softmax')(x)

model = Model(inputs=inputt, outputs=output)

model.compile(loss="categorical_crossentropy", metrics=['accuracy'], optimizer='adam')

model.summary()
history = model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.1)

NameError: name 'out_dim' is not defined

In [4]:
# 顯示訓練和驗證損失
loss = history.history["loss"]
epochs = range(1, len(loss)+1)
val_loss = history.history["val_loss"]
plt.plot(epochs, loss, "b-", label="Training Loss")
plt.plot(epochs, val_loss, "r--", label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# 顯示訓練和驗證準確度
acc = history.history["accuracy"]
epochs = range(1, len(acc)+1)
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "b-", label="Training Acc")
plt.plot(epochs, val_acc, "r--", label="Validation Acc")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# 所有圖片的路徑 "C:\workspace\python_workspace\DeepLearning\detected_face\*"
target_path = glob.glob(os.path.join(root,"Target","detected_face4","*"))
target = []
target_img = []
for img_path in target_path:
    imgg = Image.open(img_path).resize(IMG_SIZE).convert("RGB")
    target_img.append(imgg)
    img_array = np.array(imgg)
    target.append(img_array)
target = np.array(target)

plt.figure(figsize=(16,9))
count = 1
pred = model.predict(target)
# 將One hot的Y轉成1維數字陣列，數字代表其type
pred = np.argmax(pred, axis=1).astype(int)
for i,item in enumerate(target_img):
    plt.subplot(1,6,count)
    plt.title(pred[i])
    plt.axis('off')
    plt.imshow(item)
    count += 1
pred
# name_map = {"hao":0, "lo":1, "ming":2, "ting":3, "xiang":4, "yan":5}

NameError: name 'history' is not defined

In [ ]:
# 存下四人的模型及權重
model.save("4_people_model.h5",overwrite = True)

In [ ]:
X = []
Y = []
# 將每個名字都編號
name_map = {"hao":0, "lo":1, "ming":2, "ting":3, "xiang":4, "yan":5}
# root = "C:\workspace\python_workspace\DeepLearning"
root = os.getcwd()
# 所有圖片的路徑 "C:\workspace\python_workspace\DeepLearning\detected_face\*"
all_images_path = glob.glob(os.path.join(root,"detected_face6","*","*"))
for img_path in all_images_path:
    # 取得學生id
    id = img_path.split("\\")[-2]
    img = np.array(Image.open(img_path).resize(IMG_SIZE).convert("RGB"))
    X.append(img)
    Y.append(name_map[id])
X = np.array(X)
Y = np.array(Y)
Y = to_categorical(Y)
# 輸出層的結果數
out_dim = Y.shape[1]

In [ ]:
# 載入模型及權重
base_model4 = load_model("4_people_model.h5")
base_model4.summary()
# 因為改變輸出的數量，所以不取輸出層的結果
x = base_model4.layers[-2].output
# 改成理想的輸出層
output = Dense(6, activation='softmax')(x)
# 將數入層連接輸出層以建立出新的模型
model = Model(inputs=base_model4.input, outputs=output)
model.compile(loss="categorical_crossentropy", metrics=['accuracy'], optimizer='adam')

model.summary()
history = model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.1)

In [ ]:
# 顯示訓練和驗證損失
loss = history.history["loss"]
epochs = range(1, len(loss)+1)
val_loss = history.history["val_loss"]
plt.plot(epochs, loss, "b-", label="Training Loss")
plt.plot(epochs, val_loss, "r--", label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# 顯示訓練和驗證準確度
acc = history.history["accuracy"]
epochs = range(1, len(acc)+1)
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "b-", label="Training Acc")
plt.plot(epochs, val_acc, "r--", label="Validation Acc")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
# 所有圖片的路徑 "C:\workspace\python_workspace\DeepLearning\detected_face\*"
target_path = glob.glob(os.path.join(root,"Target","detected_face6","*"))
target = []
target_img = []
for img_path in target_path:
    imgg = Image.open(img_path).resize(IMG_SIZE).convert("RGB")
    target_img.append(imgg)
    img_array = np.array(imgg)
    target.append(img_array)
target = np.array(target)

plt.figure(figsize=(16,9))
count = 1
pred = model.predict(target)
# 將One hot的Y轉成1維數字陣列，數字代表其type
pred = np.argmax(pred, axis=1).astype(int)
for i,item in enumerate(target_img):
    plt.subplot(1,6,count)
    plt.title(pred[i])
    plt.axis('off')
    plt.imshow(item)
    count += 1
pred
# name_map = {"hao":0, "lo":1, "ming":2, "ting":3, "xiang":4, "yan":5}

In [ ]:
model.save("6_people_model.h5",overwrite = True)
model.save_weights("6_people_weights.h5",overwrite = True)